In [2]:
from __future__ import division, unicode_literals

import pandas as pd
import numpy as np

from sklearn.metrics import roc_auc_score

In [3]:
train = pd.read_csv('train.csv', encoding='utf-8', 
                    lineterminator='\n')

In [4]:
test = pd.read_csv('test.csv', encoding='utf-8', 
                   lineterminator='\n')

In [5]:
assert(all(train.columns[:-1] == test.columns))
assert(train.columns[-1] == 'retweet_count')

In [6]:
for index, field in enumerate(train.columns):
    print(field)

id
text
in_reply_to_user_id
user.id
user.name
user.screen_name
user.description
user.location
user.lang
user.time_zone
user.utc_offset
user.statuses_count
user.followers_count
user.friends_count
user.favourites_count
user.created_at
user.geo_enabled
user.is_translation_enabled
user.listed_count
retweet_count


In [7]:
train.head()

,id,text,in_reply_to_user_id,user.id,user.name,user.screen_name,user.description,user.location,user.lang,user.time_zone,user.utc_offset,user.statuses_count,user.followers_count,user.friends_count,user.favourites_count,user.created_at,user.geo_enabled,user.is_translation_enabled,user.listed_count,retweet_count
0,629692043326062592,Me and ma fwends 🍎 http://t.co/B3YJ31hZuc,0.0,133840449,Paul C. wilson,xcaptainpaulx,Guitarist for @chunknocaptainc // Founder @off...,NaN,fr,Paris,7200.0,1606,9164,205,758,Fri Apr 16 19:33:58 +0000 2010,True,False,27,12
1,629692041362968576,@SinedioMD @AspyrMedia Try this link https://t...,7629232.0,21245956,Direct2Drive.com,Direct2Drive,D2D has an ever-expanding library of downloada...,"California, USA",en,Pacific Time (US & Canada),-25200.0,3004,7484,708,44,Wed Feb 18 21:58:04 +0000 2009,False,False,330,0
2,629692040679419904,@davidrobots @GamesRadar @David_H_Esq @CatGone...,16535125.0,63785369,Anthony John Agnello,ajohnagnello,I am the Senior Social Editor at GamesRadar+ a...,New York City,en,Eastern Time (US & Canada),-14400.0,3988,2050,491,422,Fri Aug 07 17:49:40 +0000 2009,False,False,70,0
3,629692040666812416,Already hearing scanner traffic from Heath ask...,0.0,28812126,Bethany Bruner,bethany_bruner,"Breaking News Reporter for @NewarkAdvocate, @...",NaN,en,Eastern Time (US & Canada),-14400.0,9018,919,309,79,Sat Apr 04 15:31:07 +0000 2009,True,False,38,0
4,629692038842306560,@MelJohnson6527 next year with @Chuck_Ellis #c...,181232379.0,44024059,Nolan Smith,NdotSmitty,2010 Duke National Champ! I'm Still ON A MISSI...,DMV/Durham/Louisville/ Next??,en,Quito,-18000.0,32527,85356,912,137,Tue Jun 02 01:16:08 +0000 2009,True,False,1232,1


In [25]:
lang_rt = train[['user.lang', 'retweet_count']].groupby(by='user.lang').mean()
lang_rt = lang_rt.to_dict()
# print(lang_rt)

In [23]:
# location_rt = train[['user.location', 'retweet_count']].groupby(by='user.location').mean()
# location_rt = location_rt.to_dict()
# print(location_rt)

In [24]:
time_zone_rt = train[['user.time_zone', 'retweet_count']].groupby(by='user.time_zone').mean()
time_zone_rt = time_zone_rt.to_dict()
# print(time_zone_rt)

In [ ]:
def get_tweet_features(data):
    mentions = []
    is_citations = []
    links = []
    RLRTs = []
    is_replys = []
    for tweet in data:
        mention_c = tweet['text'].count('@')
        is_citation = tweet['text'].find('MT @') + 1
        
        if is_citation > 0:
            mention_c -= 1
            is_citation = 1
        else:
            is_citation = 0

        if tweet['in_reply_to_user_id'] != 0:
            is_replys.append(1)
            mention_c -= 1
        else:
            is_replys.append(0)
        
        if mention_c < 0:
            mention_c = 0
            
        mentions.append(mention_c)
        is_citations.append(is_citation)
        links.append(tweet['text'].count('http'))
        RLRTs.append(tweet['text'].count('RLRT'))
    features = [pd.Series.from_array(mentions),
                pd.Series.from_array(is_citations),
                pd.Series.from_array(links),
                pd.Series.from_array(RLRTs),
                pd.Series.from_array(is_replys)]
    return features

In [9]:
def df2features(df):
    return np.array([
        df['user.id'],
        df['user.followers_count'],
    ]).transpose()

train_X = df2features(train)
test_X = df2features(test)

Вычисляем значения целевого признака в обучающей выборке.

In [10]:
train_y = train['retweet_count'] > 20

Для примера в качестве модели строим простейшее решающее дерево.

In [11]:
from sklearn.tree import DecisionTreeClassifier

est = DecisionTreeClassifier()

est.fit(train_X, train_y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

Пример вычисления площади под ROC-кривой на **обучающей** выборке.

In [12]:
proba = est.predict_proba(train_X)
print(roc_auc_score(train_y, proba[:,1]))

0.989581404961


In [13]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

Читаем демо-файл с прогнозом

In [14]:
prediction = pd.read_csv('sample_prediction.csv')

prediction.head()

,id,probability
0,629692042952765440,0.765988
1,629692042717855745,0.160099
2,629692039974813696,0.825652
3,629692038242566145,0.495118
4,629692036879413248,0.656044


Строим новый прогноз и обновляем вероятности того, что ретвитов больше 20.

In [15]:
proba = est.predict_proba(test_X)
prediction['probability'] = proba[:,1]

Записываем полученный результат в файл **prediction.csv**.

In [16]:
prediction.to_csv('prediction.csv', index=False)

In [16]:
prediction.head()

,id,probability
0,629692042952765440,0.0
1,629692042717855745,0.0
2,629692039974813696,0.2
3,629692038242566145,1.0
4,629692036879413248,0.0
